In [1]:
#pip install pydub

In [2]:
#pip install yt-dlp

In [3]:
#!apt-get install ffmpeg

In [4]:
#pip install pytube

In [5]:
#pip install noisereduce

In [6]:
#pip install datasets

In [7]:
#pip install jiwer

In [8]:
from pydub import AudioSegment
from pydub.effects import normalize
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torchaudio
import os
from pytube import YouTube
from moviepy.editor import VideoFileClip
import subprocess
import noisereduce as nr
import numpy as np
import scipy.io.wavfile as wav
import librosa
import scipy.signal
from datasets import load_dataset, load_metric
import jiwer

In [9]:
import subprocess

# Path ke file audio
audio_file_path = 'lucu.wav'

# Fungsi untuk mendapatkan bitrate menggunakan ffprobe
def get_audio_bitrate(audio_file_path):
    result = subprocess.run(
        ['ffprobe', '-v', 'error', '-show_entries', 'format=bit_rate', '-of', 'default=noprint_wrappers=1:nokey=1', audio_file_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    return result.stdout.decode('utf-8').strip()

# Mendapatkan bitrate
bitrate = get_audio_bitrate(audio_file_path)
print(f"Bitrate dari file {audio_file_path}: {bitrate} bits per second")

Bitrate dari file lucu.wav:  bits per second


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# Load model dan processor
model_path = "/content/drive/MyDrive/Skripsi/wav2vec2-profanity-javanese-sby"
#model_path = "indonesian-nlp/wav2vec2-indonesian-javanese-sundanese"
model = Wav2Vec2ForCTC.from_pretrained(model_path)
processor = Wav2Vec2Processor.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# **Scraping**

In [ ]:
import yt_dlp
import subprocess

# URL video YouTube
youtube_url = "https://www.youtube.com/shorts/Lvl_EsR0og4"

# Fungsi untuk mengunduh audio menggunakan yt-dlp
def download_audio(youtube_url, output_path):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,  # Path file output sementara
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',  # Format audio yang diinginkan (WAV)
            'preferredquality': '320',  # Kualitas audio output (opsional)
        }],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

# Path file output sementara dan output akhir
temp_audio_path = 'downloaded_audio'
audio_output_path = 'output_audio.wav'

# Unduh audio dari YouTube
download_audio(youtube_url, temp_audio_path)

# Meningkatkan kualitas audio menggunakan ffmpeg
subprocess.run([
    'ffmpeg', '-i', temp_audio_path,
    '-ar', '44100', '-ac', '2', '-b:a', '1411k', audio_output_path
])

print("Audio telah berhasil diunduh dan dikonversi ke format WAV dengan kualitas optimal.")

[youtube] Extracting URL: https://www.youtube.com/shorts/Lvl_EsR0og4
[youtube] Lvl_EsR0og4: Downloading webpage
[youtube] Lvl_EsR0og4: Downloading ios player API JSON
[youtube] Lvl_EsR0og4: Downloading android player API JSON


[youtube] Lvl_EsR0og4: Downloading player 018e9916
[youtube] Lvl_EsR0og4: Downloading m3u8 information
[info] Lvl_EsR0og4: Downloading 1 format(s): 251
[download] Destination: downloaded_audio
[download] 100% of  525.71KiB in 00:00:00 at 4.25MiB/s   
[ExtractAudio] Destination: downloaded_audio.wav
Deleting original file downloaded_audio (pass -k to keep)
Audio telah berhasil diunduh dan dikonversi ke format WAV dengan kualitas optimal.


# **Modeling**

In [12]:
import librosa
import noisereduce as nr
import soundfile as sf

# Memuat file audio
file_audio_path = 'audio_1.wav'
y, sr = librosa.load(file_audio_path, sr=16000)

# Mengurangi noise
reduced_noise = nr.reduce_noise(y=y, sr=sr)

# Menyimpan file audio hasil noise reduction
output_file_path = 'audio_reduced_noise.wav'
sf.write(output_file_path, reduced_noise, sr)

print(f"Hasil noise reduction disimpan di: {output_file_path}")
# Fungsi untuk memuat dan preprocess audio
def load_and_preprocess_audio(file_path):
    audio_array, sampling_rate = torchaudio.load(file_path)

    # Resample audio jika sampling rate tidak 16000
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        audio_array = resampler(audio_array)

    # Volume normalization
    volume_transform = torchaudio.transforms.Vol(gain=1.0, gain_type='amplitude')
    audio_array = volume_transform(audio_array)

    audio_array = audio_array.squeeze().numpy()
    return audio_array

# Path ke file audio
audio_path = "/content/audio_reduced_noise.wav"

Hasil noise reduction disimpan di: audio_reduced_noise.wav


In [13]:
# Muat dan preprocess audio
audio_array = load_and_preprocess_audio(audio_path)

# Preprocess audio dengan processor
inputs = processor(audio_array, sampling_rate=16000, return_tensors="pt", padding=True)

# Model inferensi
with torch.no_grad():
    logits = model(inputs.input_values).logits

# Ambil argmax dari logits untuk mendapatkan prediksi ID
predicted_ids = torch.argmax(logits, dim=-1)

In [14]:
# Dekode prediksi ID menjadi teks
transcription = processor.batch_decode(predicted_ids)[0]

print("Transkripsi:", transcription)

Transkripsi: seng gudu mbola koni gae njawab pertanyaan ngeri pas leparan koen kudu menjawab dengan baik rama dan sopan kapen rabi jancok krame kan cok jancok ra ba koen kudu menjawab dengan ngegas didu ono lek awakmu ngamu benwongi wedi kapan lonusuula koen kudu mengalihkan perhatian kerjo munah embi saiki lo gacok penapakan ji cok ndedi la penlambakan pakean seape sribu kiro kiro koen cik seeles dadi mbngapil belappak aokno sekg gurukno kiro kiro koen buduk tik surunan farat opo maneh cules nang gomenter hah


In [15]:

def evaluate_wer(predictions, references):
    error = jiwer.wer(references, predictions)
    return error

In [16]:
# Fungsi untuk memuat dan preprocess audio
def load_and_preprocess_audio1(file_path):
    audio_array, sampling_rate = torchaudio.load(file_path)
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        audio_array = resampler(audio_array)
    audio_array = audio_array.squeeze().numpy()
    return audio_array
# Muat dan preprocess audio
audio_array1 = load_and_preprocess_audio1('audio_1.wav')

# Preprocess audio dengan processor
inputs = processor(audio_array1, sampling_rate=16000, return_tensors="pt", padding=True)

# Model inferensi
with torch.no_grad():
    logits = model(inputs.input_values).logits

# Ambil argmax dari logits untuk mendapatkan prediksi ID
predicted_ids1 = torch.argmax(logits, dim=-1)
# Dekode prediksi ID menjadi teks
transcription1 = processor.batch_decode(predicted_ids1)[0]

print("Transkripsi:", transcription1)

Transkripsi: sen gudu mbok la koen igae njawab pertanyaan ngeri pas lebaran koen gudu menjawab dengan baik ramah dan sopan kapen rabi jancok kra mee cancok cancok ra bek koen kudumen jawab dengan ngegas dudu kono oleh awakmu ngamuk baen uwongi wedi kapan lolus ukoen kuduk mengalih kan perhatian kerjo munah endi sa iki lo jancok penampakan bi cok ndene la penambahkan pakean setese ribuk kiro kiro koen tib s terus ga dikongkercol penoko konk ke urukno k kiro koro koen buduk tip surunon fokat opo maneh tules nang komenter e


In [17]:
model_path = "indonesian-nlp/wav2vec2-indonesian-javanese-sundanese"
model = Wav2Vec2ForCTC.from_pretrained(model_path)
processor = Wav2Vec2Processor.from_pretrained(model_path)
# Fungsi untuk memuat dan preprocess audio
def load_and_preprocess_audio1(file_path):
    audio_array, sampling_rate = torchaudio.load(file_path)
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        audio_array = resampler(audio_array)
    audio_array = audio_array.squeeze().numpy()
    return audio_array
# Muat dan preprocess audio
audio_array2 = load_and_preprocess_audio1('audio_1.wav')

# Preprocess audio dengan processor
inputs = processor(audio_array2, sampling_rate=16000, return_tensors="pt", padding=True)

# Model inferensi
with torch.no_grad():
    logits = model(inputs.input_values).logits

# Ambil argmax dari logits untuk mendapatkan prediksi ID
predicted_ids2 = torch.argmax(logits, dim=-1)
# Dekode prediksi ID menjadi teks
transcription2 = processor.batch_decode(predicted_ids2)[0]

print("Transkripsi:", transcription2)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

Some weights of the model checkpoint at indonesian-nlp/wav2vec2-indonesian-javanese-sundanese were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model tha

Transkripsi: sing gudu molakoni kainjawab pertanyaan ngeripas leparankonkudu menjawab dengan baik ramah dan sopan kapan rabi canck pramere cack cacukrab kon kudu menjawab dengan ngegas dudu an oleh awakmu engamu benvrb wongi wedi kapan loluskonkuduh mengalihkan perhatian kerja munandikecacok penapakan bicuk janilah menamakan pagaian satu seribukegarokontiekirkron buduh tup surnon faktapemaneh tulis langgomenter


In [18]:
predictions1 = [transcription1]
predictions = [transcription]
predictions2 = [transcription2]
references = ["seng kudu mbok lakoni gae njawab pertanyaan ngeri pas lebaran koen kudu menjawab dengan baik ramah dan sopan kapan rabi jancok rame ae jancok jancok kabeh koen kudu menjawab dengan ngegas duduono lek awakmu ngamuk ben wonge wedi kapan lulus koen kudu mengalihkan perhatian kerjomu nang ndi saiki lo jancok penampakan endi cok ini lah penempakan paket seratus ribu kiro kiro koen tips kiro kiro koen butuh tips bermanfaat opo maneh tulis nang komentar ah"]
#references = ["seng kudu mbok lakoni pas ketemu wong mokel koen kudu negur wonge kandanono iku perbuatan elek dancok anake sucipto mokel nggateli lo lo lo gak bahaya ta opo se oy gerang gableg brewoken mokel oy deloken rek mokel nang kebone cak syarul dukun ngisin ngisini warga kaliasin dan tegal sari raimu ojok rekam rekama ngono pengen oleh bojo ayu kelakuan mu ngene rabi ambek wet blimbing koen mengganggu privasi aku yo due prestasi tak rekam balik gak ngurus gak ngurus anake sucipto goblok sucipto babah deloken rek wong iki ngganggu privasi ne wong liyo"]

In [19]:

print("Transkripsi:", transcription)
wer = evaluate_wer(predictions, references)
print("Word Error Rate:", wer)

print("Transkripsi:", transcription1)
wer1 = evaluate_wer(predictions1, references)
print("Word Error Rate (without prep):", wer1)

print("Transkripsi:", transcription1)
wer2 = evaluate_wer(predictions2, references)
print("Word Error Rate (Model Wirawan):", wer2)

Transkripsi: seng gudu mbola koni gae njawab pertanyaan ngeri pas leparan koen kudu menjawab dengan baik rama dan sopan kapen rabi jancok krame kan cok jancok ra ba koen kudu menjawab dengan ngegas didu ono lek awakmu ngamu benwongi wedi kapan lonusuula koen kudu mengalihkan perhatian kerjo munah embi saiki lo gacok penapakan ji cok ndedi la penlambakan pakean seape sribu kiro kiro koen cik seeles dadi mbngapil belappak aokno sekg gurukno kiro kiro koen buduk tik surunan farat opo maneh cules nang gomenter hah
Word Error Rate: 0.5866666666666667
Transkripsi: sen gudu mbok la koen igae njawab pertanyaan ngeri pas lebaran koen gudu menjawab dengan baik ramah dan sopan kapen rabi jancok kra mee cancok cancok ra bek koen kudumen jawab dengan ngegas dudu kono oleh awakmu ngamuk baen uwongi wedi kapan lolus ukoen kuduk mengalih kan perhatian kerjo munah endi sa iki lo jancok penampakan bi cok ndene la penambahkan pakean setese ribuk kiro kiro koen tib s terus ga dikongkercol penoko konk ke u

# **Percobaan Evaluasi Data Real**

In [75]:
import librosa
import noisereduce as nr
import soundfile as sf
import torchaudio
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import jiwer

# Load model dan processor
model_path = "/content/drive/MyDrive/Skripsi/wav2vec2-profanity-javanese-sby"
#model_path = "indonesian-nlp/wav2vec2-indonesian-javanese-sundanese"
model = Wav2Vec2ForCTC.from_pretrained(model_path)
processor = Wav2Vec2Processor.from_pretrained(model_path)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [76]:
# Fungsi untuk mengurangi noise dan menyimpan hasil
def reduce_noise(input_path, output_path):
    y, sr = librosa.load(input_path, sr=16000)
    reduced_noise = nr.reduce_noise(y=y, sr=sr)
    sf.write(output_path, reduced_noise, sr)
    print(f"Hasil noise reduction disimpan di: {output_path}")

# Fungsi untuk memuat dan preprocess audio
def load_and_preprocess_audio(file_path):
    audio_array, sampling_rate = torchaudio.load(file_path)
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        audio_array = resampler(audio_array)
    volume_transform = torchaudio.transforms.Vol(gain=1.0, gain_type='amplitude')
    audio_array = volume_transform(audio_array)
    audio_array = audio_array.squeeze().numpy()
    return audio_array

# Fungsi untuk memuat audio tanpa preprocessing
def load_audio(file_path):
    audio_array, sampling_rate = torchaudio.load(file_path)
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        audio_array = resampler(audio_array)
    audio_array = audio_array.squeeze().numpy()
    return audio_array

# Fungsi untuk melakukan inferensi dan transkripsi
def transcribe_audio(audio_array):
    inputs = processor(audio_array, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

# Fungsi untuk menghitung WER
def evaluate_wer(predictions, references):
    error = jiwer.wer(references, predictions)
    return error

In [ ]:
 'audio_3.wav', 'audio_4.wav', 'audio_5.wav'
 "wah jancok subhanallah pakuwon mall bagus sekali wah hancok pakuwon guede cok sumpah yaallah deloken guede sumpah gak onok ngene iki nang sidoarjo cok jancok ye aku wes nang pakuwon mall ye aku wes nang pakuwon mall disini sangat berbeda dengan ramayan bunmurasih semunya terlihat mewah dan mahal aku disini merasa miskin buanget cok aku sampai tidak mampu membeli makanan atau minuman disini lambungku yang gembel ini terbiasa makan sego taboan ga onok sego pecel a iki yo goblok pisan nang pakuwon gowo rongewu cok goblok muliyo kono cok",
    "wah jancok kurasa aku sedang melihat sebuah keindahan alam berupa flyover aloha yang sudah bisa diakses alhamdulillah aku terkagum sampai selebrasi ssuuu numpak becak nang pandaan lo lo lo flyover e wes dadi aku jadi semakin bangga menjadi penduduk sidoarjo aku berjanji",
    "seng kudu mbok lakoni pas ketemu wong kakean basa basi peda taek jancok koen kudu ngiyani tawarane los no ae gak usah sungkan salahe nawari cok gak nge es a mas o yo gelem lawong suroboyo puanas monggo maem ngge purun pak tepak dereng maem kulo beyuh nggatel ayo mas yo ayo mas tepak gabut aku nang endi mas cok kate nang endi tuku sampo mampir sek kene lo ngadem ngadem yo endi kopine cok ndue tamu dijarno ngelak angin senggel maren kiro kiro koen butuh tips bermanfaat opo maneh tulisen nang komentar obat pahit ae nambani mosok kamu seng manis malah ngelarani"

In [77]:
# List file audio
audio_files = ['audio_1.wav', 'audio_5.wav',]

# Referensi transkripsi (gunakan transkripsi referensi yang sesuai)
references = [
    "seng kudu mbok lakoni gae njawab pertanyaan ngeri pas lebaran koen kudu menjawab dengan baik ramah dan sopan kapan rabi jancok rame ae jancok jancok kabeh koen kudu menjawab dengan ngegas duduono lek awakmu ngamuk ben wonge wedi kapan lulus koen kudu mengalihkan perhatian kerjomu nang ndi saiki lo jancok penampakan endi cok ini lah penempakan paket seratus ribu kiro kiro koen tips kiro kiro koen butuh tips bermanfaat opo maneh tulis nang komentar ah",
    "seng kudu mbok lakoni pas ketemu wong kakean basa basi peda taek jancok koen kudu ngiyani tawarane los no ae gak usah sungkan salahe nawari cok gak nge es a mas o yo gelem lawong suroboyo puanas monggo maem ngge purun pak tepak dereng maem kulo beyuh nggatel ayo mas yo ayo mas tepak gabut aku nang endi mas cok kate nang endi tuku sampo mampir sek kene lo ngadem ngadem yo endi kopine cok ndue tamu dijarno ngelak angin senggel maren kiro kiro koen butuh tips bermanfaat opo maneh tulisen nang komentar obat pahit ae nambani mosok kamu seng manis malah ngelarani"
]

In [78]:
# Proses semua file audio
predictions = []
predictions_no_prep = []

for file_audio_path in audio_files:
    # Proses dengan noise reduction dan volume normalization
    reduced_audio_path = file_audio_path.replace('.wav', '_reduced_noise.wav')
    reduce_noise(file_audio_path, reduced_audio_path)
    audio_array = load_and_preprocess_audio(reduced_audio_path)
    transcription = transcribe_audio(audio_array)
    predictions.append(transcription)

    # Proses tanpa preprocessing
    audio_array_no_prep = load_audio(file_audio_path)
    transcription_no_prep = transcribe_audio(audio_array_no_prep)
    predictions_no_prep.append(transcription_no_prep)

# Hitung WER
wer = evaluate_wer(predictions, references)
wer_no_prep = evaluate_wer(predictions_no_prep, references)

# Print hasil
print("Transkripsi dengan preprocessing:", predictions)
print("Transkripsi tanpa preprocessing:", predictions_no_prep)
print("Word Error Rate dengan preprocessing:", wer)
print("Word Error Rate tanpa preprocessing:", wer_no_prep)

Hasil noise reduction disimpan di: audio_1_reduced_noise.wav
Hasil noise reduction disimpan di: audio_5_reduced_noise.wav
Transkripsi dengan preprocessing: ['seng gudu mbola koni gae njawab pertanyaan ngeri pas leparan koen kudu menjawab dengan baik rama dan sopan kapen rabi jancok krame kan cok jancok ra ba koen kudu menjawab dengan ngegas didu ono lek awakmu ngamu benwongi wedi kapan lonusuula koen kudu mengalihkan perhatian kerjo munah embi saiki lo gacok penapakan ji cok ndedi la penlambakan pakean seape sribu kiro kiro koen cik seeles dadi mbngapil belappak aokno sekg gurukno kiro kiro koen buduk tik surunan farat opo maneh cules nang gomenter hah', 'seng iku dumboh lakoni pas ketemu wong kakean asap asing gataek cok koen kudu ngi jaki tawarane wasno ae gausaksunga salahe taware cok gak ngesa ma wo yo kelel longsr bae panas banggel maem ga porul pek tepak garek maeng bu lo du ilu gadem hek wan yo ae yo maes tepak gabut aku gandi mas cok gatelaendi tuku sampo mampir sek kele lo nga

In [79]:
predictions

['seng gudu mbola koni gae njawab pertanyaan ngeri pas leparan koen kudu menjawab dengan baik rama dan sopan kapen rabi jancok krame kan cok jancok ra ba koen kudu menjawab dengan ngegas didu ono lek awakmu ngamu benwongi wedi kapan lonusuula koen kudu mengalihkan perhatian kerjo munah embi saiki lo gacok penapakan ji cok ndedi la penlambakan pakean seape sribu kiro kiro koen cik seeles dadi mbngapil belappak aokno sekg gurukno kiro kiro koen buduk tik surunan farat opo maneh cules nang gomenter hah',
 'seng iku dumboh lakoni pas ketemu wong kakean asap asing gataek cok koen kudu ngi jaki tawarane wasno ae gausaksunga salahe taware cok gak ngesa ma wo yo kelel longsr bae panas banggel maem ga porul pek tepak garek maeng bu lo du ilu gadem hek wan yo ae yo maes tepak gabut aku gandi mas cok gatelaendi tuku sampo mampir sek kele lo ngadenganmu ya endi opine gah cok nde tamu gak elok angin segel kemaren kiro kiro akon buduti rulman fato wo mal dedulese nang komentar obat baeta nambandi mo

#**Percobaan sesungguhnya**

In [ ]:
# Kamus kata-kata vulgar (contoh)
vulgar_words = {"asu", "jancok", "cok","goblog","nggateli",
                "nggatel","dancok","hancok","ancok","ngentot",
                "taek","goblok","tolol","kontol","jangkrek",
                "silit","bajingan","anjing","jembut","longor"}
# Fungsi untuk mendeteksi kata kasar
def detect_profanity(text, profanity_list):
    words = text.split()
    profane_words = [word for word in words if word.lower() in profanity_list]
    return profane_words, words

# Fungsi untuk memperkirakan waktu kemunculan kata dalam audio
def get_profanity_timestamps(profane_words, words, audio_duration):
    profanity_timestamps = []
    word_duration = audio_duration / len(words)  # Durasi rata-rata tiap kata

    # Gunakan set untuk menyimpan kata-kata kasar yang sudah terdeteksi
    detected_set = set()

    for word in profane_words:
        if word.lower() not in detected_set:
            word_indices = [i for i, w in enumerate(words) if w.lower() == word]
            for idx in word_indices:
                word_time = idx * word_duration
                profanity_timestamps.append((word, word_time))
            # Tambahkan kata ke set untuk menandai bahwa sudah terdeteksi
            detected_set.add(word.lower())

    return profanity_timestamps

In [ ]:
# Deteksi kata-kata vulgar
detected_vulgar_words, words = detect_profanity(transcription, vulgar_words)

# Dapatkan durasi audio
audio_segment = AudioSegment.from_file(audio_path)
audio_duration = len(audio_segment) / 1000.0  # Durasi dalam detik

In [ ]:
detected_vulgar_words

['jancok', 'cok', 'jancok', 'cok']

In [ ]:
# Deteksi waktu munculnya kata-kata vulgar dalam audio
profanity_timestamps = get_profanity_timestamps(detected_vulgar_words, words, audio_duration)
print("Waktu munculnya kata-kata vulgar dalam detik:")
for word, time_seconds in profanity_timestamps:
    print(f"'{word}' pada detik ke-{time_seconds:.2f}")

Waktu munculnya kata-kata vulgar dalam detik:
'jancok' pada detik ke-10.46
'jancok' pada detik ke-12.55
'cok' pada detik ke-12.03
'cok' pada detik ke-27.72
